전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


유니크수

In [5]:
ratings['user_id'].nunique()

6039

In [6]:
ratings['movie_id'].nunique()

3628

인기작30

In [7]:
new_df = pd.merge(ratings, movies, on='movie_id')
movie_count = new_df.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

선호영화

In [8]:
# 선호영화
my_favorite = [
    'Matrix, The (1999)',
    'Forrest Gump (1994)',
    'Men in Black (1997)',
    'Groundhog Day (1993)',
    'Terminator, The (1984)'
]

# 데이터 만들기
my_rating = pd.DataFrame({'user_id' : ['무'] * 5, 'title' : my_favorite, 'count' : [5] * 5})

# user_id에 데이터가 없으면 위에 임의로 만든 my_favorite 데이터를 추가해줍니다.
if not new_df.isin({'user_id' : ['무']})['user_id'].any():
    new_df = new_df.append(my_rating)

new_df.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,무,NaN,5,NaN,"Matrix, The (1999)",NaN
1,무,NaN,5,NaN,Forrest Gump (1994),NaN
2,무,NaN,5,NaN,Men in Black (1997),NaN
3,무,NaN,5,NaN,Groundhog Day (1993),NaN
4,무,NaN,5,NaN,"Terminator, The (1984)",NaN


In [9]:
del new_df['movie_id'], new_df['genre'], new_df['timestamp']
df = pd.merge(new_df, movies, on=['title'])
df.head(10)

,user_id,count,title,movie_id,genre
0,1,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama
1,2,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama
2,12,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama
3,15,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama
4,17,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama
5,18,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama
6,19,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama
7,24,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama
8,28,3,One Flew Over the Cuckoo's Nest (1975),1193,Drama
9,33,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama


In [10]:
df[df['user_id'] == '무']

,user_id,count,title,movie_id,genre
106998,무,5,Groundhog Day (1993),1265,Comedy|Romance
124654,무,5,"Matrix, The (1999)",2571,Action|Sci-Fi|Thriller
154346,무,5,Forrest Gump (1994),356,Comedy|Romance|War
168507,무,5,Men in Black (1997),1580,Action|Adventure|Comedy|Sci-Fi
193102,무,5,"Terminator, The (1984)",1240,Action|Sci-Fi|Thriller


In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = df['user_id'].unique()
movie_unique = df['title'].unique()

# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}

# 인덱싱 제대로 되었는지 확인
print(user_to_idx['무'])
print(movie_to_idx['Modulations (1998)'])

5993
3620


In [14]:
# indexing을 통해 데이터 칼럼 내 값을 바꾸는 코드

temp_user_data = df['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(df):
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')
    
temp_movie_data = df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df):
    print('movie column indexing ok!!')
    df['movie_id2'] = temp_movie_data
else:
    print('movie column indexing fail')

df.head(10)

user_id column indexing OK!!
movie column indexing ok!!


,user_id,count,title,movie_id,genre,movie_id2
0,0,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
1,1,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
2,2,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
3,3,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
4,4,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
5,5,4,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
6,6,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
7,7,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
8,8,3,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0
9,9,5,One Flew Over the Cuckoo's Nest (1975),1193,Drama,0


CSR matrix

In [15]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['movie_id2'].nunique()

print(type(num_user), num_movie)

<class 'int'> 3628


In [16]:
csr_data = csr_matrix(
    (df['count'], (df.user_id, df.movie_id2)),
    shape = (num_user, num_movie)
)
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

als_model = AlternatingLeastSquares 모델 훈련

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 내용과는 무관하다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(
    factors = 100,
    regularization = 0.01,
    use_gpu = False,
    iterations=15,
    dtype = np.float32
)

In [19]:
# als모델은 Input으로 반대가 되기 때문에 transpose를 해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델이 예측한 나의 선호도

In [22]:
무, pulpfiction = user_to_idx['무'], movie_to_idx['Pulp Fiction (1994)']
무_vector, pulpfiction_vector = als_model.user_factors[무], als_model.item_factors[pulpfiction]

print(무_vector)

[ 0.11696295  0.50964946 -0.46415606  0.36256513 -0.34962812 -0.08255932
  0.10458025 -0.50986433  0.9834942  -0.58495235 -0.16888282 -0.14982441
 -0.5578775   0.22601084  0.5693763   0.15185174  0.05516683  0.65022343
  0.31683484  0.5328408  -0.5791579  -0.59647346 -1.2453583  -1.1643581
  0.36165053  0.21128194 -0.06971386  0.4760667  -0.602902    0.54352266
 -0.24849465 -1.0477018  -0.3301856  -0.3902305  -0.27090305  0.25136554
  0.09892706  1.6639006   0.24701247 -0.0763529  -0.8368292  -0.5184851
  1.4305822  -0.10306217  0.18237802 -0.1514235  -0.52345914 -0.588394
  0.34471312 -0.07655729 -0.24863821 -0.1772511  -0.1855768  -0.2620038
 -0.08902071  0.22618715  0.28095916  0.15655549  0.16417944  0.59075135
 -0.1065212  -0.32531655  0.18918207 -0.00414299 -0.05972171 -0.05891821
  0.02903092 -0.27827913  0.40430075  0.04697255  0.8205207   0.21180758
  0.5084436   0.17363061  0.68316907  0.3753852  -0.51944256 -0.29226786
  0.74438953  0.96693176 -0.22012964 -0.8773929   0.7642

In [23]:
print(pulpfiction_vector)

[-0.00572201  0.00092842 -0.00179988  0.00905871  0.00154179  0.00827832
 -0.00803584  0.0076652   0.02553289 -0.0075059   0.01452539 -0.00543269
 -0.00396762 -0.00087645  0.02361289  0.00635807 -0.01156515  0.01588671
 -0.00788681 -0.00299877 -0.00991319  0.01367184  0.03947967 -0.01647203
 -0.02051885  0.01948184 -0.0307198  -0.00626669 -0.00867904  0.00210841
 -0.00201878  0.00718504 -0.021853    0.01534465  0.01244057  0.01273239
  0.01018119  0.01231989 -0.01269366  0.00305804  0.01331957  0.01746472
  0.01232477 -0.00589438  0.02218474 -0.00437643 -0.01292615 -0.00030765
  0.01071993  0.02967178  0.02357927 -0.0231991   0.00601732  0.00055764
  0.01190081  0.009832    0.01974863  0.00482707  0.00741236  0.02474662
  0.04401449 -0.011355   -0.00195151  0.0200909  -0.01414051 -0.01934144
  0.02257527  0.00437669  0.01956389 -0.00413776  0.0175115   0.01845527
 -0.00092165  0.03047116 -0.02173757  0.04016278  0.01032343  0.00788755
  0.01952286  0.01995627 -0.01512678  0.02011428  0

In [25]:
np.dot(무_vector, pulpfiction_vector)

0.008711107

In [26]:
무, pulpfiction = user_to_idx['무'], movie_to_idx['Mission to Mars (2000)']
무_vector, pulpfiction_vector = als_model.user_factors[무], als_model.item_factors[pulpfiction]
np.dot(무_vector, pulpfiction_vector)

0.023906706

sf도 추천률이 낮네....

추천받기

In [33]:
user = user_to_idx['무']

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
movie_recommended

[(92, 0.6353497),
 (107, 0.4839427),
 (62, 0.4371012),
 (64, 0.37039688),
 (141, 0.3525898),
 (22, 0.3211799),
 (150, 0.31602722),
 (40, 0.31497508),
 (117, 0.30773702),
 (44, 0.28882942),
 (651, 0.2572996),
 (236, 0.25386864),
 (38, 0.2503879),
 (48, 0.24718195),
 (87, 0.24077535),
 (121, 0.23427534),
 (5, 0.23357582),
 (255, 0.22285834),
 (60, 0.21587288),
 (145, 0.2151241)]

In [37]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [40]:
favorite_movie = 'Back to the Future (1985)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple
get_similar_movie('Back to the Future (1985)')
idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Fugitive, The (1993)',
 'Back to the Future (1985)',
 'Independence Day (ID4) (1996)',
 'Toy Story (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Aliens (1986)',
 'Speed (1994)',
 'Sixth Sense, The (1999)',
 'Saving Private Ryan (1998)',
 'Braveheart (1995)',
 'Silence of the Lambs, The (1991)',
 'Princess Bride, The (1987)',
 "There's Something About Mary (1998)",
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Fifth Element, The (1997)']

꽤 맞음.